# Science Case #1:

Find all <source-type> within a given area of the sky with emission from
<molecule(s)> detected.


The science motivation might be to find all young stellar objects (YSOs) in Taurus with ALMA
detections of CO J=2-1. The return from this query would include images of the moment maps of
CO emission, peak intensity, resolution, noise, correlator setting, and other relevant information
from any Taurus YSOs (as identified by science keywords, abstract text, SIMBAD coordinate
matches) in the publicly accessible ALMA Science Archive. The user could identify which data
are appropriate for their study an pull the u,v data or image cubes of interest from the archive.

Optionally, the user could then create a query to retrieve the full set of science products for all
sources, or the sources of interest. This would allow exploration of continuum and other lines
detected in the same observations. The quantitative data are returned as tables in the user’s
Python environment (for example Jupyter notebook) which can then be manipulated (see Section
4), for example to produce a plot of continuum flux versus CO integrated intensity or CO
intensity versus [13CO/12CO] intensity ratio.



In [1]:
from astroquery.admit import ADMIT
import pandas as pd
# display the whole table inthe notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',-1)


Found  /home/mpound/src/astroumd/study7/query/admit.db
Checking db.... 0


/tmp/ipykernel_1181954/1782088392.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


In [2]:
a = ADMIT()
a.check()
#result = a.query(formula="CO", transition="2-1")
result=a.query(source_name_alma="NGC3049")

Found  /home/mpound/src/astroumd/study7/query/admit.db
Checking db.... 0
header    : 0 entries
alma      : 40 entries
win       : 40 entries
lines     : 24 entries
sources   : 84 entries
cont      : 0 entries
kwargs  {'source_name_alma': 'NGC3049'}
select * from alma, win,lines,sources WHERE alma.target_name='NGC3049'


In [ ]:
#result=a.query(source_name_alma="NGC3409")

In [49]:
result=a.sql("select * from alma,win,lines,sources where target_name='NGC3049' and win.a_id = alma.id and lines.w_id = win.id and sources.w_id = win.id")

In [50]:
len(result)

2

In [51]:
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,2,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.0432,23.9308,0.0,0.0,0.0,2,3,0,148.708594,9.271231,0.0686,0.115,0.075,0.027,134.7,3.4
1,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,2,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.0432,23.9308,0.0,0.0,0.0,1,3,0,148.709559,9.268488,0.0690,0.049,0.027,0.020,89.2,3.4


In [43]:
result=a.sql("select * from alma,win,lines where target_name='NGC3049' and win.a_id = alma.id and lines.w_id = win.id")

In [44]:
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,2,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,241.577407,2,2,241.585261,1.703099,1447.989431,1,0,109,0.000167,0.000103,0.000071,37.569119,0.027523,1,2,H2COH+,"4(1,4)-3(1,3)",243.48293,-15.7261,22.9371,0.0,0.0,0.0
1,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.834820,1.703100,1447.989431,1,2,109,0.000180,0.000102,0.000070,38.894366,0.045872,2,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.0432,23.9308,0.0,0.0,0.0
2,6,uid://A001/X1288/Xba2,NGC3049,148.70695,9.271113,243.816538,6,6,243.824357,1.703119,1447.989431,2,0,109,0.000234,0.000900,0.000706,50.378832,0.055046,3,6,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-20.9074,36.8238,0.0,0.0,0.0
3,6,uid://A001/X1288/Xba2,NGC3049,148.70695,9.271113,243.816538,6,6,243.824357,1.703119,1447.989431,2,0,109,0.000234,0.000900,0.000706,50.378832,0.055046,4,6,g-CH3CH2OH,"14(1,13)-13(1,12),vt=1-1",244.63395,-15.5264,22.9554,0.0,0.0,0.0


In [ ]:
result=a.sql("select * from alma,win,lines,sources where target_name='NGC3049' and win.a_id = alma.id and sources.w_id = win.id")

In [ ]:
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,1,2,H2COH+,"4(1,4)-3(1,3)",243.48293,-15.72610,22.937100,0.0,0.0,0.0,2,3,0,148.708594,9.271231,0.0686,0.115,0.075,0.027,134.7,3.4
1,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,1,2,H2COH+,"4(1,4)-3(1,3)",243.48293,-15.72610,22.937100,0.0,0.0,0.0,1,3,0,148.709559,9.268488,0.0690,0.049,0.027,0.020,89.2,3.4
2,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,2,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.04320,23.930800,0.0,0.0,0.0,2,3,0,148.708594,9.271231,0.0686,0.115,0.075,0.027,134.7,3.4
3,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,2,3,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-53.04320,23.930800,0.0,0.0,0.0,1,3,0,148.709559,9.268488,0.0690,0.049,0.027,0.020,89.2,3.4
4,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,3,6,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-20.90740,36.823800,0.0,0.0,0.0,2,3,0,148.708594,9.271231,0.0686,0.115,0.075,0.027,134.7,3.4
5,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,3,6,t-HCOOH,"2(2,0)-1(1,1)",244.59816,-20.90740,36.823800,0.0,0.0,0.0,1,3,0,148.709559,9.268488,0.0690,0.049,0.027,0.020,89.2,3.4
6,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,4,6,g-CH3CH2OH,"14(1,13)-13(1,12),vt=1-1",244.63395,-15.52640,22.955400,0.0,0.0,0.0,2,3,0,148.708594,9.271231,0.0686,0.115,0.075,0.027,134.7,3.4
7,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,4,6,g-CH3CH2OH,"14(1,13)-13(1,12),vt=1-1",244.63395,-15.52640,22.955400,0.0,0.0,0.0,1,3,0,148.709559,9.268488,0.0690,0.049,0.027,0.020,89.2,3.4
8,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,5,7,CO,2-1,230.53800,-136.84900,138.029000,0.0,0.0,0.0,2,3,0,148.708594,9.271231,0.0686,0.115,0.075,0.027,134.7,3.4
9,3,uid://A001/X1288/Xba0,NGC3049,148.70695,9.271113,243.826966,3,3,243.83482,1.7031,1447.989431,1,2,109,0.00018,0.000102,0.00007,38.894366,0.045872,5,7,CO,2-1,230.53800,-136.84900,138.029000,0.0,0.0,0.0,1,3,0,148.709559,9.268488,0.0690,0.049,0.027,0.020,89.2,3.4
